In [3]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
dataFrame = pd.read_csv('CTU-13-Dataset/9/capture20110817.binetflow')

In [5]:
dataFrame.head(5)

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/17 12:01:12.984851,0.976560,tcp,93.45.141.223,2254,->,147.32.84.118,6881,S_RA,0.0,0.0,4,244,124,flow=Background-TCP-Attempt
1,2011/08/17 12:01:15.616709,0.950668,tcp,84.16.60.37,64136,->,147.32.84.118,6881,S_RA,0.0,0.0,4,244,124,flow=Background-TCP-Attempt
2,2011/08/17 12:01:23.209772,1.006908,tcp,94.44.197.227,3844,->,147.32.84.118,6881,S_RA,0.0,0.0,4,276,156,flow=Background-TCP-Attempt
3,2011/08/17 12:01:24.216680,1.000400,tcp,94.44.197.227,3844,->,147.32.84.118,6881,S_RA,0.0,0.0,4,276,156,flow=Background-TCP-Attempt
4,2011/08/17 12:03:27.216922,1.093291,tcp,147.32.3.51,1656,->,147.32.87.22,10010,S_RA,0.0,0.0,4,244,124,flow=Background-TCP-Attempt


In [6]:
class DQNSolver():
    def __init__(self, observation_space_size, action_space_size):
        self.model = Sequential()
        self.model.add(Dense(24, input_dim=observation_space_size, activation='relu'))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(action_space_size, activation='linear'))
        self.model.compile(loss="mean_squared_error", optimizer=Adam(lr=LEARNING_RATE))
    

In [7]:
# From: https://github.com/mgarzon/cybersec/blob/master/MalwareDetection.ipynb
def preprocessData(dataFrame):

    '''
    This function is used to perform
    the necessary operations to 
    convert the raw data into a
    clean data set.

    '''
    listOfFeaturesToDrop = [
    'Dir',
    'sTos',
    'dTos']
    dataFrame = dataFrame.drop(listOfFeaturesToDrop, axis=1)

    #dp.deleteNullRow(dataFrame,'Sport')
    #dp.deleteNullRow(dataFrame,'Dport')
    #dp.deleteRowWhere(dataFrame,'Sport','x')
    #dp.deleteRowWhere(dataFrame,'Dport','x')
    
    dataFrame = dataFrame.replace({'Sport': {'x': np.nan}, 'Dport': {'x': np.nan}})
    #dataFrame = dataFrame.dropna(subset=['Sport', 'Dport'])
    dataFrame = dataFrame.dropna()
    
    # TODO
    #dp.convertColumnToInt32(dataFrame,'Sport')
    #dp.convertColumnToInt32(dataFrame,'Dport')

    #dp.convertColumnToTimeStamp(dataFrame,'StartTime') # ?? already a timestamp

    #dp.convertColumnToFloat16(dataFrame,'Dur')
    #dp.convertColumnToInt16(dataFrame,'TotPkts')

    return dataFrame

In [8]:
dataFrame = preprocessData(dataFrame)

In [9]:
dataFrame.head(5)

,StartTime,Dur,Proto,SrcAddr,Sport,DstAddr,Dport,State,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/17 12:01:12.984851,0.976560,tcp,93.45.141.223,2254,147.32.84.118,6881,S_RA,4,244,124,flow=Background-TCP-Attempt
1,2011/08/17 12:01:15.616709,0.950668,tcp,84.16.60.37,64136,147.32.84.118,6881,S_RA,4,244,124,flow=Background-TCP-Attempt
2,2011/08/17 12:01:23.209772,1.006908,tcp,94.44.197.227,3844,147.32.84.118,6881,S_RA,4,276,156,flow=Background-TCP-Attempt
3,2011/08/17 12:01:24.216680,1.000400,tcp,94.44.197.227,3844,147.32.84.118,6881,S_RA,4,276,156,flow=Background-TCP-Attempt
4,2011/08/17 12:03:27.216922,1.093291,tcp,147.32.3.51,1656,147.32.87.22,10010,S_RA,4,244,124,flow=Background-TCP-Attempt
